In [1]:
!pip install contractions

     |████████████████████████████████| 321 kB 5.3 MB/s 
     |████████████████████████████████| 284 kB 47.0 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85447 sha256=035dabcb76aa63177ecba603b405f71b1ad21cb749fbfce4b8d7a59c47542d6c
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick


In [2]:
import contractions
import nltk
nltk.download('punkt')
import string
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf
import numpy as np
import time
from google.colab import drive
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import numpy as np
import re
import math
from tqdm import tqdm
import matplotlib.pyplot as plt

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Dataset source: https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html

In [3]:
#mount google colab
drive.mount('/content/drive',force_remount = True) 
#read the data

Mounted at /content/drive


In [5]:



movie_line = "/content/drive/MyDrive/kawther_chatbot_fake/cornell_movie_dialogs_corpus/cornell movie-dialogs corpus/movie_lines.txt"
movie_convo = "/content/drive/MyDrive/kawther_chatbot_fake/cornell_movie_dialogs_corpus/cornell movie-dialogs corpus/movie_conversations.txt"

m_lines = open(movie_line , encoding='utf-8',errors='ignore').read().split('\n')
c_lines = open(movie_convo , encoding='utf-8',errors='ignore').read().split('\n')

convo_line = []
for lines in c_lines:
    _lines = lines.split(" +++$+++ ")[-1][1:-1].replace("'","").replace(" ","")
    convo_line.append(_lines.split(","))

id_line = {}
for lines in m_lines:
    _lines = lines.split(" +++$+++ ")
    if len(_lines) == 5:
        id_line[_lines[0]] = _lines[4]

questions = []
answers = []

for line in convo_line:
    for i in range(len(line) -1):
        questions.append(id_line[line[i]])
        answers.append(id_line[line[i+1]])
        
def clean_text(text):
    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"  ","",text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    
    return text

clean_questions = []
clean_answers = []

for q in questions:
    clean_questions.append(clean_text(q))
for a in answers:
    clean_answers.append(clean_text(a))


In [6]:
data = pd.DataFrame(
    {'question': clean_questions,
     'answer': clean_answers
    })

In [7]:
pd.set_option('display.max_colwidth', None)
data.head()

,question,answer
0,can we make this quickroxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quadagain,well i thought we would start with pronunciation if that is okay with you
1,well i thought we would start with pronunciation if that is okay with you,not the hacking and gagging and spitting partplease
2,not the hacking and gagging and spitting partplease,okay then how about we try out some french cuisinesaturdaynight
3,you are asking me outthat is so cute that is your name again,forget it
4,no no it is my fault we did not have a proper introduction,cameron


In [8]:
data = data[:10000]
data

#we have taken only 10000 rows from the whole corpus. the first 10000. 

,question,answer
0,can we make this quickroxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quadagain,well i thought we would start with pronunciation if that is okay with you
1,well i thought we would start with pronunciation if that is okay with you,not the hacking and gagging and spitting partplease
2,not the hacking and gagging and spitting partplease,okay then how about we try out some french cuisinesaturdaynight
3,you are asking me outthat is so cute that is your name again,forget it
4,no no it is my fault we did not have a proper introduction,cameron
...,...,...
9995,sergeant west's served for twenty three years he is the public face of the modern army,and you notice i am not in the army anymore
9996,gotta be honest i love what you have done with the place,you and the colonel go back
9997,you and the colonel go back,he got me through basic and a lot of other stuff i owe him
9998,he got me through basic and a lot of other stuff i owe him,you are the tom hardy i have been reading about in the papers right new orleans pd fired you for taking bribes from guissepe torres


In [9]:
#to download the 10K rows 
"""from google.colab import files

# e.g. save pandas output as csv
data.to_csv('example_10K.txt')

# or any other file as usual
# with open('example.csv', 'w') as f:
#   f.write('your strings here')

files.download('example_10K.txt')
"""

"from google.colab import files\n\n# e.g. save pandas output as csv\ndata.to_csv('example_10K.txt')\n\n# or any other file as usual\n# with open('example.csv', 'w') as f:\n#   f.write('your strings here')\n\nfiles.download('example_10K.txt')\n"

In [10]:
data['question_tokenized'] = data['question'].apply(lambda x: [contractions.fix(word) for word in str(x).split()])
data['answer_tokenized'] = data['answer'].apply(lambda x: [contractions.fix(word) for word in str(x).split()])
data.head()

,question,answer,question_tokenized,answer_tokenized
0,can we make this quickroxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quadagain,well i thought we would start with pronunciation if that is okay with you,"[can, we, make, this, quickroxanne, korrine, and, andrew, barrett, are, having, an, incredibly, horrendous, public, break, up, on, the, quadagain]","[well, i, thought, we, would, start, with, pronunciation, if, that, is, okay, with, you]"
1,well i thought we would start with pronunciation if that is okay with you,not the hacking and gagging and spitting partplease,"[well, i, thought, we, would, start, with, pronunciation, if, that, is, okay, with, you]","[not, the, hacking, and, gagging, and, spitting, partplease]"
2,not the hacking and gagging and spitting partplease,okay then how about we try out some french cuisinesaturdaynight,"[not, the, hacking, and, gagging, and, spitting, partplease]","[okay, then, how, about, we, try, out, some, french, cuisinesaturdaynight]"
3,you are asking me outthat is so cute that is your name again,forget it,"[you, are, asking, me, outthat, is, so, cute, that, is, your, name, again]","[forget, it]"
4,no no it is my fault we did not have a proper introduction,cameron,"[no, no, it, is, my, fault, we, did, not, have, a, proper, introduction]",[cameron]


In [11]:
data['question_tokenized'] = [' '.join(map(str, l)) for l in data['question_tokenized']]
data['answer_tokenized'] = [' '.join(map(str, l)) for l in data['answer_tokenized']]
data.head()

,question,answer,question_tokenized,answer_tokenized
0,can we make this quickroxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quadagain,well i thought we would start with pronunciation if that is okay with you,can we make this quickroxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quadagain,well i thought we would start with pronunciation if that is okay with you
1,well i thought we would start with pronunciation if that is okay with you,not the hacking and gagging and spitting partplease,well i thought we would start with pronunciation if that is okay with you,not the hacking and gagging and spitting partplease
2,not the hacking and gagging and spitting partplease,okay then how about we try out some french cuisinesaturdaynight,not the hacking and gagging and spitting partplease,okay then how about we try out some french cuisinesaturdaynight
3,you are asking me outthat is so cute that is your name again,forget it,you are asking me outthat is so cute that is your name again,forget it
4,no no it is my fault we did not have a proper introduction,cameron,no no it is my fault we did not have a proper introduction,cameron


In [12]:
data['question_tokenized'] = data['question_tokenized'].apply(nltk.word_tokenize)
data['answer_tokenized'] = data['answer_tokenized'].apply(nltk.word_tokenize)
data.head()

,question,answer,question_tokenized,answer_tokenized
0,can we make this quickroxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quadagain,well i thought we would start with pronunciation if that is okay with you,"[can, we, make, this, quickroxanne, korrine, and, andrew, barrett, are, having, an, incredibly, horrendous, public, break, up, on, the, quadagain]","[well, i, thought, we, would, start, with, pronunciation, if, that, is, okay, with, you]"
1,well i thought we would start with pronunciation if that is okay with you,not the hacking and gagging and spitting partplease,"[well, i, thought, we, would, start, with, pronunciation, if, that, is, okay, with, you]","[not, the, hacking, and, gagging, and, spitting, partplease]"
2,not the hacking and gagging and spitting partplease,okay then how about we try out some french cuisinesaturdaynight,"[not, the, hacking, and, gagging, and, spitting, partplease]","[okay, then, how, about, we, try, out, some, french, cuisinesaturdaynight]"
3,you are asking me outthat is so cute that is your name again,forget it,"[you, are, asking, me, outthat, is, so, cute, that, is, your, name, again]","[forget, it]"
4,no no it is my fault we did not have a proper introduction,cameron,"[no, no, it, is, my, fault, we, did, not, have, a, proper, introduction]",[cameron]


In [13]:
data['question_tokenized'] = data['question_tokenized'].apply(lambda x: [word.lower() for word in x])
data['answer_tokenized'] = data['answer_tokenized'].apply(lambda x: [word.lower() for word in x])
data.head()

,question,answer,question_tokenized,answer_tokenized
0,can we make this quickroxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quadagain,well i thought we would start with pronunciation if that is okay with you,"[can, we, make, this, quickroxanne, korrine, and, andrew, barrett, are, having, an, incredibly, horrendous, public, break, up, on, the, quadagain]","[well, i, thought, we, would, start, with, pronunciation, if, that, is, okay, with, you]"
1,well i thought we would start with pronunciation if that is okay with you,not the hacking and gagging and spitting partplease,"[well, i, thought, we, would, start, with, pronunciation, if, that, is, okay, with, you]","[not, the, hacking, and, gagging, and, spitting, partplease]"
2,not the hacking and gagging and spitting partplease,okay then how about we try out some french cuisinesaturdaynight,"[not, the, hacking, and, gagging, and, spitting, partplease]","[okay, then, how, about, we, try, out, some, french, cuisinesaturdaynight]"
3,you are asking me outthat is so cute that is your name again,forget it,"[you, are, asking, me, outthat, is, so, cute, that, is, your, name, again]","[forget, it]"
4,no no it is my fault we did not have a proper introduction,cameron,"[no, no, it, is, my, fault, we, did, not, have, a, proper, introduction]",[cameron]


In [14]:
punc = string.punctuation
data['question_tokenized'] = data['question_tokenized'].apply(lambda x: [word for word in x if word not in punc])
data.head()

,question,answer,question_tokenized,answer_tokenized
0,can we make this quickroxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quadagain,well i thought we would start with pronunciation if that is okay with you,"[can, we, make, this, quickroxanne, korrine, and, andrew, barrett, are, having, an, incredibly, horrendous, public, break, up, on, the, quadagain]","[well, i, thought, we, would, start, with, pronunciation, if, that, is, okay, with, you]"
1,well i thought we would start with pronunciation if that is okay with you,not the hacking and gagging and spitting partplease,"[well, i, thought, we, would, start, with, pronunciation, if, that, is, okay, with, you]","[not, the, hacking, and, gagging, and, spitting, partplease]"
2,not the hacking and gagging and spitting partplease,okay then how about we try out some french cuisinesaturdaynight,"[not, the, hacking, and, gagging, and, spitting, partplease]","[okay, then, how, about, we, try, out, some, french, cuisinesaturdaynight]"
3,you are asking me outthat is so cute that is your name again,forget it,"[you, are, asking, me, outthat, is, so, cute, that, is, your, name, again]","[forget, it]"
4,no no it is my fault we did not have a proper introduction,cameron,"[no, no, it, is, my, fault, we, did, not, have, a, proper, introduction]",[cameron]


In [15]:
def clean(x):
  for i,word in enumerate(x):
    wordNew = re.sub('[\-_,.`"\']', '', word)
    if len(wordNew)<1:
      x.remove(word)
    elif word != wordNew:
      x[i] = wordNew
  return x

data['question_tokenized'].apply(lambda x: clean(x))
data['answer_tokenized'].apply(lambda x: clean(x))
data.head()

,question,answer,question_tokenized,answer_tokenized
0,can we make this quickroxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quadagain,well i thought we would start with pronunciation if that is okay with you,"[can, we, make, this, quickroxanne, korrine, and, andrew, barrett, are, having, an, incredibly, horrendous, public, break, up, on, the, quadagain]","[well, i, thought, we, would, start, with, pronunciation, if, that, is, okay, with, you]"
1,well i thought we would start with pronunciation if that is okay with you,not the hacking and gagging and spitting partplease,"[well, i, thought, we, would, start, with, pronunciation, if, that, is, okay, with, you]","[not, the, hacking, and, gagging, and, spitting, partplease]"
2,not the hacking and gagging and spitting partplease,okay then how about we try out some french cuisinesaturdaynight,"[not, the, hacking, and, gagging, and, spitting, partplease]","[okay, then, how, about, we, try, out, some, french, cuisinesaturdaynight]"
3,you are asking me outthat is so cute that is your name again,forget it,"[you, are, asking, me, outthat, is, so, cute, that, is, your, name, again]","[forget, it]"
4,no no it is my fault we did not have a proper introduction,cameron,"[no, no, it, is, my, fault, we, did, not, have, a, proper, introduction]",[cameron]


In [16]:
data.head(50)

,question,answer,question_tokenized,answer_tokenized
0,can we make this quickroxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quadagain,well i thought we would start with pronunciation if that is okay with you,"[can, we, make, this, quickroxanne, korrine, and, andrew, barrett, are, having, an, incredibly, horrendous, public, break, up, on, the, quadagain]","[well, i, thought, we, would, start, with, pronunciation, if, that, is, okay, with, you]"
1,well i thought we would start with pronunciation if that is okay with you,not the hacking and gagging and spitting partplease,"[well, i, thought, we, would, start, with, pronunciation, if, that, is, okay, with, you]","[not, the, hacking, and, gagging, and, spitting, partplease]"
2,not the hacking and gagging and spitting partplease,okay then how about we try out some french cuisinesaturdaynight,"[not, the, hacking, and, gagging, and, spitting, partplease]","[okay, then, how, about, we, try, out, some, french, cuisinesaturdaynight]"
3,you are asking me outthat is so cute that is your name again,forget it,"[you, are, asking, me, outthat, is, so, cute, that, is, your, name, again]","[forget, it]"
4,no no it is my fault we did not have a proper introduction,cameron,"[no, no, it, is, my, fault, we, did, not, have, a, proper, introduction]",[cameron]
5,cameron,the thing is cameron i am at the mercy of a particularly hideous breed of losermy sisteri cannot date until she does,[cameron],"[the, thing, is, cameron, i, am, at, the, mercy, of, a, particularly, hideous, breed, of, losermy, sisteri, can, not, date, until, she, does]"
6,the thing is cameron i am at the mercy of a particularly hideous breed of losermy sisteri cannot date until she does,seems like she could get a date easy enough,"[the, thing, is, cameron, i, am, at, the, mercy, of, a, particularly, hideous, breed, of, losermy, sisteri, can, not, date, until, she, does]","[seems, like, she, could, get, a, date, easy, enough]"
7,why,unsolved mysteryshe used to be really popular when she started high school then it was just like she got sick of it or something,[why],"[unsolved, mysteryshe, used, to, be, really, popular, when, she, started, high, school, then, it, was, just, like, she, got, sick, of, it, or, something]"
8,unsolved mysteryshe used to be really popular when she started high school then it was just like she got sick of it or something,that is a shame,"[unsolved, mysteryshe, used, to, be, really, popular, when, she, started, high, school, then, it, was, just, like, she, got, sick, of, it, or, something]","[that, is, a, shame]"
9,gosh if only we could find kat a boyfriend,let me see what i can do,"[gosh, if, only, we, could, find, kat, a, boyfriend]","[let, me, see, what, i, can, do]"


In [17]:
word_count = {}
for question in data['question_tokenized']:
    for word in question:
        if word not in word_count:
            word_count[word] = 1
        else:
            word_count[word] += 1
for answer in data['answer_tokenized']:
    for word in answer:
        if word not in word_count:
            word_count[word] = 1
        else:
            word_count[word] += 1
print(len(word_count))
print(word_count)

13341
{'can': 1118, 'we': 1679, 'make': 271, 'this': 1465, 'quickroxanne': 1, 'korrine': 1, 'and': 2665, 'andrew': 1, 'barrett': 1, 'are': 2512, 'having': 50, 'an': 445, 'incredibly': 8, 'horrendous': 1, 'public': 17, 'break': 37, 'up': 727, 'on': 1185, 'the': 6135, 'quadagain': 1, 'well': 563, 'i': 8156, 'thought': 186, 'would': 1025, 'start': 77, 'with': 1141, 'pronunciation': 2, 'if': 791, 'that': 3023, 'is': 5247, 'okay': 232, 'you': 9211, 'not': 4834, 'hacking': 4, 'gagging': 2, 'spitting': 2, 'partplease': 2, 'asking': 32, 'me': 2132, 'outthat': 1, 'so': 784, 'cute': 6, 'your': 1494, 'name': 129, 'again': 104, 'no': 1109, 'it': 3732, 'my': 1508, 'fault': 23, 'did': 896, 'have': 2303, 'a': 4781, 'proper': 9, 'introduction': 1, 'cameron': 10, 'thing': 261, 'am': 1813, 'at': 679, 'mercy': 4, 'of': 2651, 'particularly': 5, 'hideous': 2, 'breed': 2, 'losermy': 2, 'sisteri': 2, 'date': 33, 'until': 69, 'she': 540, 'does': 291, 'why': 579, 'unsolved': 2, 'mysteryshe': 2, 'used': 69, 'to

feature engineering 

In [18]:
threshold= 2
words_indexed = {}
word_number = 4

tokens = ['<PAD>', '<OUT>', '<SOS>', '<EOS>']
for token in tokens:
    words_indexed[token] = len(words_indexed)

for word, count in word_count.items():
    if count >= threshold:
        words_indexed[word] = word_number
        word_number += 1

sizeOfVocab = len(words_indexed)
print(sizeOfVocab)
print(words_indexed)


8413
{'<PAD>': 0, '<OUT>': 1, '<SOS>': 2, '<EOS>': 3, 'can': 4, 'we': 5, 'make': 6, 'this': 7, 'and': 8, 'are': 9, 'having': 10, 'an': 11, 'incredibly': 12, 'public': 13, 'break': 14, 'up': 15, 'on': 16, 'the': 17, 'well': 18, 'i': 19, 'thought': 20, 'would': 21, 'start': 22, 'with': 23, 'pronunciation': 24, 'if': 25, 'that': 26, 'is': 27, 'okay': 28, 'you': 29, 'not': 30, 'hacking': 31, 'gagging': 32, 'spitting': 33, 'partplease': 34, 'asking': 35, 'me': 36, 'so': 37, 'cute': 38, 'your': 39, 'name': 40, 'again': 41, 'no': 42, 'it': 43, 'my': 44, 'fault': 45, 'did': 46, 'have': 47, 'a': 48, 'proper': 49, 'cameron': 50, 'thing': 51, 'am': 52, 'at': 53, 'mercy': 54, 'of': 55, 'particularly': 56, 'hideous': 57, 'breed': 58, 'losermy': 59, 'sisteri': 60, 'date': 61, 'until': 62, 'she': 63, 'does': 64, 'why': 65, 'unsolved': 66, 'mysteryshe': 67, 'used': 68, 'to': 69, 'be': 70, 'really': 71, 'popular': 72, 'when': 73, 'started': 74, 'high': 75, 'school': 76, 'then': 77, 'was': 78, 'just': 7

In [19]:
for question in data['question_tokenized']:
    question.append('<EOS>')
    question.insert(0,'<SOS>')

for answer in data['answer_tokenized']:
    answer.append('<EOS>')
    answer.insert(0,'<SOS>')

In [20]:
print(data['question_tokenized'])

0                  [<SOS>, can, we, make, this, quickroxanne, korrine, and, andrew, barrett, are, having, an, incredibly, horrendous, public, break, up, on, the, quadagain, <EOS>]
1                                                                            [<SOS>, well, i, thought, we, would, start, with, pronunciation, if, that, is, okay, with, you, <EOS>]
2                                                                                                        [<SOS>, not, the, hacking, and, gagging, and, spitting, partplease, <EOS>]
3                                                                                          [<SOS>, you, are, asking, me, outthat, is, so, cute, that, is, your, name, again, <EOS>]
4                                                                                            [<SOS>, no, no, it, is, my, fault, we, did, not, have, a, proper, introduction, <EOS>]
                                                                                           ...      

In [21]:
def convertToInt(inputSen):
    global words_indexed
    sen_into_int = []
    for word in inputSen:
        if word not in words_indexed:
            sen_into_int.append(words_indexed['<OUT>'])
        else:
            sen_into_int.append(words_indexed[word])
    return sen_into_int

data['question_tokenized_int'] = data['question_tokenized'].apply(lambda x: convertToInt(x))
data['answer_tokenized_int'] = data['answer_tokenized'].apply(lambda x: convertToInt(x))

In [22]:
data.head(50)

,question,answer,question_tokenized,answer_tokenized,question_tokenized_int,answer_tokenized_int
0,can we make this quickroxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quadagain,well i thought we would start with pronunciation if that is okay with you,"[<SOS>, can, we, make, this, quickroxanne, korrine, and, andrew, barrett, are, having, an, incredibly, horrendous, public, break, up, on, the, quadagain, <EOS>]","[<SOS>, well, i, thought, we, would, start, with, pronunciation, if, that, is, okay, with, you, <EOS>]","[2, 4, 5, 6, 7, 1, 1, 8, 1, 1, 9, 10, 11, 12, 1, 13, 14, 15, 16, 17, 1, 3]","[2, 18, 19, 20, 5, 21, 22, 23, 24, 25, 26, 27, 28, 23, 29, 3]"
1,well i thought we would start with pronunciation if that is okay with you,not the hacking and gagging and spitting partplease,"[<SOS>, well, i, thought, we, would, start, with, pronunciation, if, that, is, okay, with, you, <EOS>]","[<SOS>, not, the, hacking, and, gagging, and, spitting, partplease, <EOS>]","[2, 18, 19, 20, 5, 21, 22, 23, 24, 25, 26, 27, 28, 23, 29, 3]","[2, 30, 17, 31, 8, 32, 8, 33, 34, 3]"
2,not the hacking and gagging and spitting partplease,okay then how about we try out some french cuisinesaturdaynight,"[<SOS>, not, the, hacking, and, gagging, and, spitting, partplease, <EOS>]","[<SOS>, okay, then, how, about, we, try, out, some, french, cuisinesaturdaynight, <EOS>]","[2, 30, 17, 31, 8, 32, 8, 33, 34, 3]","[2, 28, 77, 100, 237, 5, 969, 118, 535, 424, 1, 3]"
3,you are asking me outthat is so cute that is your name again,forget it,"[<SOS>, you, are, asking, me, outthat, is, so, cute, that, is, your, name, again, <EOS>]","[<SOS>, forget, it, <EOS>]","[2, 29, 9, 35, 36, 1, 27, 37, 38, 26, 27, 39, 40, 41, 3]","[2, 259, 43, 3]"
4,no no it is my fault we did not have a proper introduction,cameron,"[<SOS>, no, no, it, is, my, fault, we, did, not, have, a, proper, introduction, <EOS>]","[<SOS>, cameron, <EOS>]","[2, 42, 42, 43, 27, 44, 45, 5, 46, 30, 47, 48, 49, 1, 3]","[2, 50, 3]"
5,cameron,the thing is cameron i am at the mercy of a particularly hideous breed of losermy sisteri cannot date until she does,"[<SOS>, cameron, <EOS>]","[<SOS>, the, thing, is, cameron, i, am, at, the, mercy, of, a, particularly, hideous, breed, of, losermy, sisteri, can, not, date, until, she, does, <EOS>]","[2, 50, 3]","[2, 17, 51, 27, 50, 19, 52, 53, 17, 54, 55, 48, 56, 57, 58, 55, 59, 60, 4, 30, 61, 62, 63, 64, 3]"
6,the thing is cameron i am at the mercy of a particularly hideous breed of losermy sisteri cannot date until she does,seems like she could get a date easy enough,"[<SOS>, the, thing, is, cameron, i, am, at, the, mercy, of, a, particularly, hideous, breed, of, losermy, sisteri, can, not, date, until, she, does, <EOS>]","[<SOS>, seems, like, she, could, get, a, date, easy, enough, <EOS>]","[2, 17, 51, 27, 50, 19, 52, 53, 17, 54, 55, 48, 56, 57, 58, 55, 59, 60, 4, 30, 61, 62, 63, 64, 3]","[2, 437, 80, 63, 87, 131, 48, 61, 973, 306, 3]"
7,why,unsolved mysteryshe used to be really popular when she started high school then it was just like she got sick of it or something,"[<SOS>, why, <EOS>]","[<SOS>, unsolved, mysteryshe, used, to, be, really, popular, when, she, started, high, school, then, it, was, just, like, she, got, sick, of, it, or, something, <EOS>]","[2, 65, 3]","[2, 66, 67, 68, 69, 70, 71, 72, 73, 63, 74, 75, 76, 77, 43, 78, 79, 80, 63, 81, 82, 55, 43, 83, 84, 3]"
8,unsolved mysteryshe used to be really popular when she started high school then it was just like she got sick of it or something,that is a shame,"[<SOS>, unsolved, mysteryshe, used, to, be, really, popular, when, she, started, high, school, then, it, was, just, like, she, got, sick, of, it, or, something, <EOS>]","[<SOS>, that, is, a, shame, <EOS>]","[2, 66, 67, 68, 69, 70, 71, 72, 73, 63, 74, 75, 76, 77, 43, 78, 79, 80, 63, 81, 82, 55, 43, 83, 84, 3]","[2, 26, 27, 48, 2040, 3]"
9,gosh if only we could find kat a boyfriend,let me see what i can do,"[<SOS>, g

In [23]:
def converToTensor(question,answer):
    question = question.to_tensor()
    answer = answer.to_tensor()
    return question,answer

In [24]:
#data[data.columns[0:2]].values
#testingData=tf.ragged.constant(data[data.columns[4:6]].values)
#data

In [25]:
#print(testingData[0][1])

In [26]:
#testingOutput=[([2,5,3],[2,4,1]),([2,8,3],[2,4,3])]
#hh=tf.convert_to_tensor(testingOutput)
#print(type(hh[0]))
#testingTensor = tf.ragged.constant(testingOutput)
#print(type(testingTensor))
#print(testingTensor[0])
#datasetNew = tf.data.Dataset.from_tensor_slices(testingTensor)
#for x in testingData:
#  print(x)
#  break
#print(testingData.to_tensor())

In [27]:
#datasetNew = tf.data.Dataset.from_tensor_slices(testingData)
#tensorData = testingData.to_tensor()
#tensorData
#datasetNew = tf.data.Dataset.from_tensor_slices(tensorData)
#tensorData=tf.expand_dims(tensorData,axis=1)
#tensorData
#for x in tensorData:
  #print(tf.reshape(x, [,]))
  #print(tf.expand_dims(x,axis=0))
 # print(x)
 # break
#datasetNew = tf.reshape(testingData, [,])
#print(testingData.get_shape())

In [28]:
#print(type(data['question_tokenized_int'].values))
listOfQuestionsNpArray=data['question_tokenized_int'].values
listOfAnswersNpArray=data['answer_tokenized_int'].values
tensorQuestions=tf.ragged.constant(listOfQuestionsNpArray,dtype=tf.int64)
tensorAnswers=tf.ragged.constant(listOfAnswersNpArray,dtype=tf.int64)
datasetNew = tf.data.Dataset.from_tensor_slices((tensorQuestions,tensorAnswers))


In [29]:
def convertSentenceToTensor(sentence):
  sentence = [contractions.fix(word) for word in str(sentence).split()]
  #print(sentence)
  sentence = ' '.join(map(str, sentence))
  #print(sentence)
  sentence = str(sentence).split()
  #print(sentence)
  sentence = [word.lower() for word in sentence]
  #print(sentence)
  sentence = [word for word in sentence if word not in punc]
  #print(sentence)
  sentence = clean(sentence)
  #print(sentence)
  sentence.append('<EOS>')
  sentence.insert(0,'<SOS>')
  #print(sentence)
  sentence = convertToInt(sentence)
  #print(sentence)
  sentence = tf.convert_to_tensor(sentence, dtype=tf.int64)
  #print(sentence)
  sentence = tf.expand_dims(sentence, 0)
  print(sentence)
  return sentence

In [30]:
convertSentenceToTensor("Hi, I'm robot and my name is \"kawther\" ! ? $")

tf.Tensor([[  2 156  19  52   1   8  44  40  27   1   3]], shape=(1, 11), dtype=int64)


<tf.Tensor: shape=(1, 11), dtype=int64, numpy=array([[  2, 156,  19,  52,   1,   8,  44,  40,  27,   1,   3]])>

In [31]:
#for x in datasetNew:
#  print(x)
#  break

In [32]:

#datasetNew = tf.data.Dataset.from_tensor_slices(hh)
#print(type(datasetNew))
#for x in datasetNew:
#  print(x)
#  break

In [33]:
BUFFER_SIZE = 2500
BATCH_SIZE = 32

In [34]:
def make_batches(ds):
  return (
      ds
      .cache()
      .shuffle(BUFFER_SIZE)
      .batch(BATCH_SIZE)
      .map(converToTensor, num_parallel_calls=tf.data.AUTOTUNE)
      .prefetch(tf.data.AUTOTUNE))


train_batches = make_batches(datasetNew)

In [35]:
for x in train_batches:
  print("outB:")
  print(x)
  break

outB:
(<tf.Tensor: shape=(32, 28), dtype=int64, numpy=
array([[   2,   19,   52,   30, 2815,   29,    3,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0],
       [   2,   37, 1554,   97,   30,   47,   69,   97,  216,    1,  170,
          69,   47,  184, 1231,    7,  550,    3,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0],
       [   2,  114,  123,  269,   55,    1,   29,   47, 1144,    1,  193,
         340, 1145, 1146,  386,    9,  288,    1, 1147, 1148,    1,   26,
         120,   55,   39, 1149, 1150,    3],
       [   2,   19,  258,  304,   29,   99,  174, 1488,    3,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0],
       [   2,  726,   39,  363,  727,  128,    3,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,

In [36]:
#print(type(train_batches))
#for x in train_batches:
#  print("outPT:")
#  print(x[0][0])
#  print("outEN:")
#  print(x[1][0])
#  print("out:")
#  print(fromIntToWords(tf.expand_dims(x[0][0],axis=0)))
#  print("out2:")
#  print(fromIntToWords(tf.expand_dims(x[1][0],axis=0)))
#  break

Transformer architecture source: https://www.tensorflow.org/text/tutorials/transformer

In [37]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates
def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)


In [38]:
def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

  # add extra dimensions to add the padding
  # to the attention logits.
  return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)
def create_look_ahead_mask(size):
  mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
  return mask  # (seq_len, seq_len)

https://towardsdatascience.com/transformers-89034557de14

In [39]:
def scaled_dot_product_attention(q, k, v, mask):
  """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead)
  but it must be broadcastable for addition.

  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable
          to (..., seq_len_q, seq_len_k). Defaults to None.

  Returns:
    output, attention_weights
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output, attention_weights
def print_out(q, k, v):
  temp_out, temp_attn = scaled_dot_product_attention(
      q, k, v, None)
  print('Attention weights are:')
  print(temp_attn)
  print('Output is:')
  print(temp_out)
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)

    self.dense = tf.keras.layers.Dense(d_model)

  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]

    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)

    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

    return output, attention_weights

In [40]:
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])

In [41]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)

  def call(self, x, training, mask):

    attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

    return out2
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(DecoderLayer, self).__init__()

    self.mha1 = MultiHeadAttention(d_model, num_heads)
    self.mha2 = MultiHeadAttention(d_model, num_heads)

    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    self.dropout3 = tf.keras.layers.Dropout(rate)

  def call(self, x, enc_output, training,
           look_ahead_mask, padding_mask):
    # enc_output.shape == (batch_size, input_seq_len, d_model)

    attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)  # (batch_size, target_seq_len, d_model)
    attn1 = self.dropout1(attn1, training=training)
    out1 = self.layernorm1(attn1 + x)

    attn2, attn_weights_block2 = self.mha2(
        enc_output, enc_output, out1, padding_mask)  # (batch_size, target_seq_len, d_model)
    attn2 = self.dropout2(attn2, training=training)
    out2 = self.layernorm2(attn2 + out1)  # (batch_size, target_seq_len, d_model)

    ffn_output = self.ffn(out2)  # (batch_size, target_seq_len, d_model)
    ffn_output = self.dropout3(ffn_output, training=training)
    out3 = self.layernorm3(ffn_output + out2)  # (batch_size, target_seq_len, d_model)

    return out3, attn_weights_block1, attn_weights_block2

In [42]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding,
                                            self.d_model)

    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate)
                       for _ in range(num_layers)]

    self.dropout = tf.keras.layers.Dropout(rate)

  def call(self, x, training, mask):

    seq_len = tf.shape(x)[1]

    # adding embedding and position encoding.
    x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)

    return x  # (batch_size, input_seq_len, d_model)
class Decoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

    self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate)
                       for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(rate)

  def call(self, x, enc_output, training,
           look_ahead_mask, padding_mask):

    seq_len = tf.shape(x)[1]
    attention_weights = {}

    x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x, block1, block2 = self.dec_layers[i](x, enc_output, training,
                                             look_ahead_mask, padding_mask)

      attention_weights[f'decoder_layer{i+1}_block1'] = block1
      attention_weights[f'decoder_layer{i+1}_block2'] = block2

    # x.shape == (batch_size, target_seq_len, d_model)
    return x, attention_weights

In [43]:
class Transformer(tf.keras.Model):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               target_vocab_size, answer_input, question_target, rate=0.1):
    super(Transformer, self).__init__()

    self.encoder = Encoder(num_layers, d_model, num_heads, dff,
                             input_vocab_size, answer_input, rate)

    self.decoder = Decoder(num_layers, d_model, num_heads, dff,
                           target_vocab_size, question_target, rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)

  def call(self, inp, tar, training, enc_padding_mask,
           look_ahead_mask, dec_padding_mask):

    enc_output = self.encoder(inp, training, enc_padding_mask)  # (batch_size, inp_seq_len, d_model)

    # dec_output.shape == (batch_size, tar_seq_len, d_model)
    dec_output, attention_weights = self.decoder(
        tar, enc_output, training, look_ahead_mask, dec_padding_mask)

    final_output = self.final_layer(dec_output)  # (batch_size, tar_seq_len, target_vocab_size)

    return final_output, attention_weights

In [44]:
num_layers = 4
d_model = 128
dff = 512
num_heads = 8
dropout_rate = 0.1

In [45]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [46]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

In [47]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [48]:
def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_sum(loss_)/tf.reduce_sum(mask)


def accuracy_function(real, pred):
  accuracies = tf.equal(real, tf.argmax(pred, axis=2))

  mask = tf.math.logical_not(tf.math.equal(real, 0))
  accuracies = tf.math.logical_and(mask, accuracies)

  accuracies = tf.cast(accuracies, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(accuracies)/tf.reduce_sum(mask)

In [49]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.Mean(name='train_accuracy')

In [50]:
transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=sizeOfVocab + 1,
    target_vocab_size=sizeOfVocab + 1,
    answer_input=1000,
    question_target=1000,
    rate=dropout_rate)

In [51]:
def create_masks(inp, tar):
  # Encoder padding mask
  enc_padding_mask = create_padding_mask(inp)

  # Used in the 2nd attention block in the decoder.
  # This padding mask is used to mask the encoder outputs.
  dec_padding_mask = create_padding_mask(inp)

  # Used in the 1st attention block in the decoder.
  # It is used to pad and mask future tokens in the input received by
  # the decoder.
  look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
  dec_target_padding_mask = create_padding_mask(tar)
  combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

  return enc_padding_mask, combined_mask, dec_padding_mask

In [52]:
#checkpoint_path = "./checkpoints2/train"

checkpoint_path = "/content/drive/MyDrive/kawther_chatbot_fake/checkpoints1/train"

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print('Latest checkpoint restored!!')

Latest checkpoint restored!!


In [53]:
EPOCHS = 2 #à changer 

In [54]:
# The @tf.function trace-compiles train_step into a TF graph for faster
# execution. The function specializes to the precise shape of the argument
# tensors. To avoid re-tracing due to the variable sequence lengths or variable
# batch sizes (the last batch is smaller), use input_signature to specify
# more generic shapes.

train_step_signature = [
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),#tf.int64
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
]


@tf.function(input_signature=train_step_signature)
def train_step(inp, tar):
  tar_inp = tar[:, :-1]
  tar_real = tar[:, 1:]

  enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)

  with tf.GradientTape() as tape:
    predictions, _ = transformer(inp, tar_inp,
                                 True,
                                 enc_padding_mask,
                                 combined_mask,
                                 dec_padding_mask)
    loss = loss_function(tar_real, predictions)

  gradients = tape.gradient(loss, transformer.trainable_variables)
  optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

  train_loss(loss)
  train_accuracy(accuracy_function(tar_real, predictions))

In [ ]:
for epoch in range(EPOCHS):
  start = time.time()

  train_loss.reset_states()
  train_accuracy.reset_states()

  for (batch, (inp, tar)) in enumerate(train_batches):
    train_step(inp, tar)

    if batch % 50 == 0:
      print(f'Epoch {epoch + 1} Batch {batch} Loss {train_loss.result():.4f} Accuracy {train_accuracy.result():.4f}')

  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print(f'Saving checkpoint for epoch {epoch+1} at {ckpt_save_path}')

  print(f'Epoch {epoch + 1} Loss {train_loss.result():.4f} Accuracy {train_accuracy.result():.4f}')

  print(f'Time taken for 1 epoch: {time.time() - start:.2f} secs\n')

Epoch 1 Batch 0 Loss 1.0053 Accuracy 0.7746
Epoch 1 Batch 50 Loss 0.8304 Accuracy 0.8089
Epoch 1 Batch 100 Loss 0.8287 Accuracy 0.8098
Epoch 1 Batch 150 Loss 0.8384 Accuracy 0.8073
Epoch 1 Batch 200 Loss 0.8404 Accuracy 0.8075
Epoch 1 Batch 250 Loss 0.8341 Accuracy 0.8089
Epoch 1 Batch 300 Loss 0.8326 Accuracy 0.8096
Epoch 1 Loss 0.8326 Accuracy 0.8097
Time taken for 1 epoch: 358.39 secs

Epoch 2 Batch 0 Loss 0.6137 Accuracy 0.8701
Epoch 2 Batch 50 Loss 0.8349 Accuracy 0.8067
Epoch 2 Batch 100 Loss 0.8256 Accuracy 0.8103
Epoch 2 Batch 150 Loss 0.8304 Accuracy 0.8095
Epoch 2 Batch 200 Loss 0.8380 Accuracy 0.8078


In [ ]:
#ckpt_save_path = ckpt_manager.save()

In [ ]:
def returnTokens():
  start = tf.convert_to_tensor(2, dtype=tf.int64)
  end = tf.convert_to_tensor(3, dtype=tf.int64)
  return start, end

In [ ]:
print(returnTokens()[0])

In [ ]:
inv_words_indexed = {value:key for key, value in words_indexed.items()}
def fromIntToWords(output):
  sentence = []
  for number in output[0]:
    #print(inv_words_indexed[number])
    #with tf.Session() as sess:  print(number.eval())
    word = inv_words_indexed[number.numpy()]
    if number.numpy() not in [1,2,3]:
      sentence.append(word)
  sentence = " ".join(sentence)
  return sentence

In [ ]:
fromIntToWords(convertSentenceToTensor("Hi, this is test."))

In [ ]:
def evaluate(sentence, max_length=40):

  sentence = convertSentenceToTensor(sentence)
  encoder_input = sentence
  start, end = returnTokens()
  output = tf.convert_to_tensor([start])
  output = tf.expand_dims(output, 0)

  for i in range(max_length):
    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
        encoder_input, output)

    # predictions.shape == (batch_size, seq_len, vocab_size)
    predictions, attention_weights = transformer(encoder_input,
                                                 output,
                                                 False,
                                                 enc_padding_mask,
                                                 combined_mask,
                                                 dec_padding_mask)

    # select the last word from the seq_len dimension
    predictions = predictions[:, -1:, :]  # (batch_size, 1, vocab_size)

    predicted_id = tf.argmax(predictions, axis=-1)
    # concatentate the predicted_id to the output which is given to the decoder
    # as its input.
    output = tf.concat([output, predicted_id], axis=-1)
    # return the result if the predicted_id is equal to the end token
    if predicted_id == end:
      break

  # output.shape (1, tokens)
  text = fromIntToWords(output)  # shape: ()
  return text

In [ ]:
def print_translation(sentence, tokens):
  print(f'{"Input:":15s}: {sentence}')
  print(f'{"Prediction":15s}: {tokens}')

In [ ]:
sentence = "Hi"

translated_text = evaluate(sentence)
print_translation(sentence, translated_text)

In [ ]:
sentence = "who are you"

translated_text = evaluate(sentence)
print_translation(sentence, translated_text)

In [ ]:
sentence = "well i thought we would start with pronunciation if that is okay with you"

translated_text = evaluate(sentence)
print_translation(sentence, translated_text)

In [ ]:

sentence = "you are asking me outthat is so cute that is your name again"

translated_text = evaluate(sentence)
print_translation(sentence, translated_text)